# Light Curve Fitting with ExoCTK
ExoCTK performs light curve fitting with the `exoctk.lightcurve_fitting` tool. This notebook will show you how to do it. 

First, some imports:

In [1]:
# Imports
import numpy as np
import astropy.units as q
from exoctk.lightcurve_fitting.simulations import simulate_lightcurve
from exoctk.lightcurve_fitting.lightcurve import LightCurve
from exoctk.lightcurve_fitting.parameters import Parameters
from exoctk.lightcurve_fitting.models import PolynomialModel, TransitModel
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

And now let's generate a fake lightcurve with real planetary parameters. We'll use WASP-19b as our example:

In [7]:
# WASP-19b lightcurve
time, flux, sim_params = simulate_lightcurve('WASP-19b')
unc = flux/100.

## Create a light curve
Creating a light curve instance is simple. Just pass `exoctk.lightcurve_fitting.lightcurve.LightCurve()` a `time` and `flux` array. You can also pass it the associated uncertainty with the `unc` argument.

The `units` argument is just the units of the given time axis, in this case 'day'.

In [12]:
# Instantiate a LightCurve object
lc = LightCurve(time, flux, unc, name='Data')
lc.plot()

## Create some models to fit to the light curve
Now that we have our light curve data loaded, let's create some models to fit to it.

First we'll create a simple linear model by passing some polynomial coefficients to the `PolynomialModel` class.

In [13]:
# Create a PolynomialModel instance with coeffs [c0, c1]. Also give it a name and format for fun.
lin_model = PolynomialModel(c1=0.01, c0=1., name='linear', fmt='g--')

# Plot it
lin_model.plot(time, draw=True)

Let's also create a transit model with the `ExoCTK.lightcurve_fitting.models.TransitModel` class. We can pass the arguments `rp`, `per`, `t0`, etc. directly to the `TransitModel` instance, or we can create a `ExoCTK.lightcurve_fitting.parameters.Parameters` instance first like so:

In [14]:
# Set the intial parameters
params = Parameters()
params.rp = sim_params['rp'], 'free', 0.1, 0.3
params.per = sim_params['per'], 'fixed'
params.t0 = sim_params['t0'], 'free', 55708., 55709.
params.inc = sim_params['inc'], 'free', 80., 90.
params.a = sim_params['a'], 'free', 1., 5.
params.ecc = sim_params['ecc'], 'fixed'
params.w = sim_params['w'], 'fixed'
params.limb_dark = sim_params['limb_dark'], 'independent'
params.transittype = sim_params['transittype'], 'independent'
params.u1 = sim_params['u'][0], 'free', 0., 0.3
params.u2 = sim_params['u'][1], 'free', 0., 0.3

# Make the transit model
t_model = TransitModel(parameters=params, name='transit', fmt='r--')

# Plot it
t_model.plot(time, draw=True)

An arbitrary number of models can then be multiplied to produce a `ExoCTK.lightcurve_fitting.models.CompositeModel` to fit to the data.

In [15]:
# Make a new model by multiplying some model components (which don't necessarily overlap)
comp_model = t_model*lin_model
comp_model.name = 'composite'

# Plot it
comp_model.plot(time, components=True, draw=True)

## Fit the model to the light curve data
To fit a model to the data, we need to supply the `LightCurve.fit()` method with a `Model` instance and then specify our choice of fitting routine. Here we'll use the `lmfit` fitter with our composite model create above.

In [16]:
# Create a new model instance from the best fit parameters
lc.fit(comp_model, fitter='lmfit', method='powell')

# Plot it
lc.plot()

[[Model]]
    Model(eval)
[[Fit Statistics]]
    # fitting method   = Powell
    # function evals   = 97
    # data points      = 1000
    # variables        = 8
    chi-square         = 11.4496698
    reduced chi-square = 0.01154201
    Akaike info crit   = -4453.79439
    Bayesian info crit = -4414.53235
[[Variables]]
    u1:   0.26689055 (init = 0.1)
    per:  0.788839 (fixed)
    ecc:  0.002 (fixed)
    u2:   0.26689055 (init = 0.1)
    t0:   55708.9920 (init = 55708.03)
    rp:   0.29166489 (init = 0.1430457)
    w:    90 (fixed)
    inc:  80.7469808 (init = 90)
    c1:   2.59792896 (init = 0.01)
    c0:   3.58792896 (init = 1)
    a:    3.57945256 (init = 3.513625)

